# Segmentation for Keyence

pycodex version 0.1.3

In [1]:
import os

import tensorflow as tf
from pycodex.data_process import crop
from pycodex import io, metadata, markerim, utils, visualization as vis
from tifffile import tifffile

# GPU setup: Specify visible GPU(s) and allow memory growth
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

try:
    gpus = tf.config.list_physical_devices("GPU")
    if gpus:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"Using GPU(s): {[gpu.name for gpu in gpus]}")
    else:
        print("No GPU detected, using CPU.")
except RuntimeError as e:
    print(f"GPU setup failed: {e}")


Using GPU(s): ['/physical_device:GPU:0']


# 00. Data Overview

In [2]:
marker_dir = "/mnt/nfs/storage/RCC/RCC_formal_CODEX/RCC_TMA543/images/final"
metadata_dict = io.organize_metadata_keyence(marker_dir)

unique_markers, _, _, _ = metadata.summary_markers(metadata_dict)
utils.display_items(unique_markers)
utils.display_items(list(metadata_dict.keys()))


Summary of Markers:
- Total unique markers: 77
- Unique markers: 75 ['ATP5A', 'C1Q', 'CA9_250ms', 'CA9_500ms', 'CD11b', 'CD11c', 'CD138', 'CD16', 'CD163', 'CD20', 'CD28', 'CD31', 'CD3e', 'CD4', 'CD45', 'CD45RA', 'CD45RO', 'CD56', 'CD57', 'CD68', 'CD69', 'CD8', 'CD86', 'Ch1Cy1', 'Ch1Cy10', 'Ch1Cy11', 'Ch1Cy12', 'Ch1Cy13', 'Ch1Cy14', 'Ch1Cy15', 'Ch1Cy16', 'Ch1Cy17', 'Ch1Cy18', 'Ch1Cy19', 'Ch1Cy2', 'Ch1Cy20', 'Ch1Cy21', 'Ch1Cy22', 'Ch1Cy23', 'Ch1Cy24', 'Ch1Cy25', 'Ch1Cy28', 'Ch1Cy3', 'Ch1Cy30', 'Ch1Cy31', 'Ch1Cy4', 'Ch1Cy5', 'Ch1Cy6', 'Ch1Cy7', 'Ch1Cy8', 'Ch1Cy9', 'Cytokeratin', 'DIG_TREM2', 'FoxP3', 'G6PD', 'GLUT1', 'GranzymeB', 'HLA1', 'IDO-1', 'IFN-y', 'Ki-67', 'LAG-3', 'MPO', 'NaKATP', 'P53', 'PD-1', 'PDL1_100_500ms', 'PDL1_50_250ms', 'Podoplanin', 'T-bet', 'TCF1_7', 'Tim-3', 'Tox_Tox2', 'VDAC1', 'aSMA']
- Blank markers: 2 ['Blank', 'BLANK_500ms_1']
- Markers duplicated in some regions: 0 []
- Markers missing in some regions: 0 []


,1,2,3,4,5,6,7,8,9,10
0,ATP5A,C1Q,CA9_250ms,CA9_500ms,CD11b,CD11c,CD138,CD16,CD163,CD20
1,CD28,CD31,CD3e,CD4,CD45,CD45RA,CD45RO,CD56,CD57,CD68
2,CD69,CD8,CD86,Ch1Cy1,Ch1Cy10,Ch1Cy11,Ch1Cy12,Ch1Cy13,Ch1Cy14,Ch1Cy15
3,Ch1Cy16,Ch1Cy17,Ch1Cy18,Ch1Cy19,Ch1Cy2,Ch1Cy20,Ch1Cy21,Ch1Cy22,Ch1Cy23,Ch1Cy24
4,Ch1Cy25,Ch1Cy28,Ch1Cy3,Ch1Cy30,Ch1Cy31,Ch1Cy4,Ch1Cy5,Ch1Cy6,Ch1Cy7,Ch1Cy8
5,Ch1Cy9,Cytokeratin,DIG_TREM2,FoxP3,G6PD,GLUT1,GranzymeB,HLA1,IDO-1,IFN-y
6,Ki-67,LAG-3,MPO,NaKATP,P53,PD-1,PDL1_100_500ms,PDL1_50_250ms,Podoplanin,T-bet
7,TCF1_7,Tim-3,Tox_Tox2,VDAC1,aSMA,,,,,


,1,2,3,4,5,6,7,8,9,10
0,reg001,reg002,reg003,reg004,reg005,reg006,reg007,reg008,reg009,reg010
1,reg011,reg012,reg013,reg014,reg015,reg016,reg017,reg018,reg019,reg020
2,reg021,reg022,reg023,reg024,reg025,,,,,
